In [5]:
%load_ext autoreload
%autoreload 2

In [12]:
import argparse
import random
import json

import numpy as np
import pandas as pd

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

from data_loader import DataGenerator
from tqdm import tqdm
from transformers import BertConfig
from bert_utils import SmilesBertModel

In [19]:
# Create Data Generator
train_iterable = DataGenerator(
    data_json_path=f'data/test.json',
    k=5,
    repr="smiles_only",
)

In [20]:
batch = next(iter(train_iterable))

In [24]:
pred = torch.zeros((1,2))
l = torch.zeros((1,2))

In [29]:
l[0,1] = 1
pred[0,1] = 1

In [30]:
pred.eq(l).sum().item() / (
    1 * 2
)

1.0

In [32]:
print(pred, l)

tensor([[0., 1.]]) tensor([[0., 1.]])


In [31]:
loss_fn = nn.BCEWithLogitsLoss()
loss_fn(pred, l)

tensor(0.5032)

In [10]:


k = 5
batch_size = 3

# Create Data Generator
train_iterable = DataGenerator(
    data_json_path=f'data/train.json',
    k=k,
    repr="smiles_only",
)
train_loader = iter(
    torch.utils.data.DataLoader(
        train_iterable,
        batch_size=batch_size,
        num_workers=1,
        pin_memory=True,
    )
)

repr_to_input_dims = {"smiles_only": 2 + 767, 
                        "concat": 2 + 767 + 640,
                        "concat_smiles_vaeprot": 2 + 767 + 100}


In [11]:
model_config = BertConfig(
    max_position_embeddings = (k+1)*2,
    hidden_size = 2 + 767,
    num_hidden_layers = 4,
    num_attention_heads = 1,
    intermediate_size = 64,
    classifier_dropout = 0.3,
    attention_probs_dropout_prob = 0.3,
    hidden_dropout_prob = 0.3,
    k = 5,
    batch_size = 3,
)

# Create model
model = SmilesBertModel(model_config)
# model.to(device)

# Create optimizer
optim = torch.optim.Adam(model.parameters(), lr=1e-5)

attention_mask = torch.ones((batch_size, (k+1)*2))

for step in tqdm(range(100)):
    i, l = next(train_loader)
    loss = model(i.float(), l.float(), attention_mask)
    loss.backward()
    optim.step()
    optim.zero_grad()

    if (step+1) % 10 == 0: 
        print(loss)

  0%|          | 0/100 [00:00<?, ?it/s]


torch.Size([3, 2, 2])


Exception: 